### historic data grab

In [2]:
import pandas as pd

directory = '../../Parcl Labs/data/recorder/Historic_Sales/'
years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

metro_counties = [
    'Cherokee', 
    'Clayton', 
    'Cobb', 
    'DeKalb', 
    'Douglas', 
    'Fayette', 
    'Forsyth',
    'Fulton', 
    'Gwinnett', 
    'Henry', 
    'Rockdale'
    ]

master_df = []

for year in years:

    print(f'collecting data for {year}...')
    year = pd.read_csv(directory + f'{year}_sales.csv', low_memory=False)

    # filter to only include metro counties
    year = year[year['county'].isin(metro_counties)]

    # create a new 'month-year' column that takes just the month and year from the 'sale_date' column
    year['month_year'] = pd.to_datetime(year['sale_date']).dt.to_period('M')

    year = year[['parcl_property_id', 'county', 'month_year']]

    master_df.append(year)

full_df = pd.concat(master_df, ignore_index=True)

print(f'total rows in master df: {full_df.shape[0]:,}')
full_df['county'].value_counts()

collecting data for 2013...
collecting data for 2014...
collecting data for 2015...
collecting data for 2016...
collecting data for 2017...
collecting data for 2018...
collecting data for 2019...
collecting data for 2020...
collecting data for 2021...
collecting data for 2022...
collecting data for 2023...
collecting data for 2024...
total rows in master df: 1,428,003


county
Fulton      352819
Gwinnett    231810
Cobb        207502
DeKalb      203513
Cherokee     97077
Forsyth      90864
Henry        75187
Clayton      65610
Douglas      41099
Fayette      35772
Rockdale     26750
Name: count, dtype: int64

In [3]:
summarized_df1 = full_df.groupby(['county', 'month_year']).agg(
    total_sales=('parcl_property_id', 'count')
).reset_index()

summarized_df1

,county,month_year,total_sales
0,Cherokee,2013-01,410
1,Cherokee,2013-02,419
2,Cherokee,2013-03,549
3,Cherokee,2013-04,582
4,Cherokee,2013-05,586
...,...,...,...
1579,Rockdale,2024-08,144
1580,Rockdale,2024-09,141
1581,Rockdale,2024-10,145
1582,Rockdale,2024-11,108


### get recent sales

In [4]:
df_sales = pd.read_csv('../../Parcl Labs/data/recorder/MasterRecorder_GA.csv', low_memory=False)

df_sales = df_sales[df_sales['county'].isin(metro_counties)]
df_sales = df_sales[df_sales['sale_date'] >= '2025-01-01']

df_sales['month_year'] = pd.to_datetime(df_sales['sale_date']).dt.to_period('M')

df_sales = df_sales[['parcl_property_id', 'county', 'month_year']]

summarized_df2 = df_sales.groupby(['county', 'month_year']).agg(
    total_sales=('parcl_property_id', 'count')
).reset_index()

summarized_df2

,county,month_year,total_sales
0,Cherokee,2025-01,329
1,Cherokee,2025-02,365
2,Cherokee,2025-03,513
3,Cherokee,2025-04,531
4,Cherokee,2025-05,553
...,...,...,...
138,Rockdale,2025-09,179
139,Rockdale,2025-10,188
140,Rockdale,2025-11,162
141,Rockdale,2025-12,166


In [5]:
full_df = pd.concat([summarized_df1, summarized_df2])
full_df = full_df.sort_values(by=['county', 'month_year'], ascending=True)
full_df.to_csv('base_data.csv', index=False)